# conv net

In [1]:
%matplotlib inline
import torch
from torch import nn
from torch.utils import data
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

## 1.load data

In [2]:
train_set = datasets.FashionMNIST(root='./data', train=True, transform=transforms.ToTensor(), download=True)
test_set = datasets.FashionMNIST(root='./data', train=False, transform=transforms.ToTensor())
len(train_set), len(test_set)
train_set[0][0].shape

torch.Size([1, 28, 28])

In [3]:
batch_size = 128
train_iter = data.DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=4)
test_iter = data.DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=2)

## 2.model

use vgg blocks<br>
一个具有num_convs个卷积层，输入channels为input_channels,输出channels为output_channels的vgg_block

In [4]:
def vgg_block(num_convs, input_channels, output_channels):
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(input_channels, output_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        input_channels = output_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

construct vgg network

In [5]:
net = nn.Sequential(vgg_block(2, 1, 64),
                    vgg_block(2, 64, 128),
                    nn.Flatten(),
                    nn.Linear(128 * 7 * 7, 512), nn.ReLU(), nn.Dropout(0.5),
                    nn.Linear(512, 512), nn.ReLU(), nn.Dropout(0.5),
                    nn.Linear(512, 10))
net = nn.Sequential(nn.Flatten(), nn.Linear(784, 128), nn.ReLU(), nn.Linear(128, 10))
x = torch.randn(size=(1, 1, 28, 28))
for blk in net:
    x = blk(x)
    print(x.shape)

torch.Size([1, 784])
torch.Size([1, 128])
torch.Size([1, 128])
torch.Size([1, 10])


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=0.05)

# 3.training

与fully connected net一样

In [7]:
def accuracy(data_iter, model):
    num = 0
    accurate_num = 0
    for x, y in data_iter:
        y_pred = model(x)
        accurate_num += torch.sum(torch.argmax(y_pred, dim=1) == y).item()
        num += y.shape[0]
    return accurate_num / num 

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [8]:
num_epochs = 100
net = net.to(device)
for epoch in range(num_epochs):
    l = 0.0
    for x, y in train_iter:
        x, y = x.to(device), y.to(device)
        y_pred = net(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        l += loss
    if (epoch + 1) % 5 == 0:
        print(f"epoch {epoch + 1} loss: {l}, train accuracy: {accuracy(train_iter, net)}, test accuracy: {accuracy(test_iter, net)}")

epoch 5 loss: 296.6308288574219, train accuracy: 0.6940666666666667, test accuracy: 0.6811
epoch 10 loss: 290.6116638183594, train accuracy: 0.73845, test accuracy: 0.7202
epoch 15 loss: 293.2864685058594, train accuracy: 0.7643, test accuracy: 0.7518
epoch 20 loss: 312.1535339355469, train accuracy: 0.7634, test accuracy: 0.7432
epoch 25 loss: 294.8633728027344, train accuracy: 0.7646833333333334, test accuracy: 0.7469
epoch 30 loss: 279.1613464355469, train accuracy: 0.7450333333333333, test accuracy: 0.7291
epoch 35 loss: 300.5340270996094, train accuracy: 0.74895, test accuracy: 0.7327
epoch 40 loss: 270.14532470703125, train accuracy: 0.7614, test accuracy: 0.7412
epoch 45 loss: 315.07763671875, train accuracy: 0.74265, test accuracy: 0.7223
epoch 50 loss: 277.7186584472656, train accuracy: 0.7638166666666667, test accuracy: 0.7432
epoch 55 loss: 296.4124755859375, train accuracy: 0.77085, test accuracy: 0.749
epoch 60 loss: 266.8048095703125, train accuracy: 0.7693333333333333, t

# 4.testing

In [9]:
print(accuracy(test_iter, net))

0.7445
